In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
grid = gpd.read_parquet('/data/uscuni-germany/processed_data/foreigners_2022.parquet')

In [3]:
grid = grid['geometry']

In [4]:
from shapely import box
german_boundary = box(*grid.to_crs(epsg=3035).total_bounds)
regions = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')
german_regions = regions[regions.intersects(german_boundary)].index.values

In [5]:
grid_assignment = pd.Series()

In [6]:
%%time

for region_id in german_regions:

    print(region_id)
    buildings = gpd.read_parquet(f"/data/uscuni-ulce/processed_data/clusters/{region_id}_clusters.pq", columns=['geometry', 'final_without_noise'])

    
    # assign grid cells to tessellation cells
    inp, res = grid.sindex.query(buildings.centroid, predicate='intersects')
    # buildings should be assigned to only one h3 grid
    duplicated = pd.Series(res).duplicated()
    inp = inp[~duplicated]
    res = res[~duplicated]

    # save regional results
    region_res = buildings.iloc[inp].groupby(res)['final_without_noise'].agg(lambda x: pd.Series.mode(x)[0])
    region_res.index = grid.index[region_res.index.values]
    grid_assignment = pd.concat((grid_assignment, region_res))

4


<timed exec>:17: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


10
132
134
286
313
400
523
765
801
832
913
960
1124
1154
1387
1478
1515
1605
1718
1736
1782
1970
1981
2096
2322
2350
2478
2514
2625
2728
2975
3039
3109
3150
3221
3250
3526
3610
3612
3701
3705
3752
3759
3981
4070
4214
4215
4235
4284
4356
4382
4723
4805
5096
5191
5246
5310
5408
5427
5662
5671
5766
5883
6254
6529
6560
6576
6741
6749
6811
6873
6996
7068
7094
7280
7485
7528
7534
7681
7688
7712
7727
7805
7914
7937
7963
8046
8216
8238
8256
8265
8345
8374
8396
8592
8707
8731
8757
8759
8813
9016
9064
9074
9150
9169
9194
9284
9824
9924
9954
9972
10019
10086
10095
10124
10179
10222
10263
10277
10455
10510
10511
10563
10579
10602
10666
10794
10847
10908
10926
10970
11002
11019
11057
11141
11210
11256
11261
11305
11309
11311
11318
11367
11444
11455
11471
11667
11678
11735
11757
11799
11877
11905
12027
12084
12100
12115
12154
12191
12381
12440
12483
12552
12667
12707
12755
12756
12844
12919
12965
13076
13137
13172
13191
13196
13229
13301
13395
13442
13482
13506
13553
13555
13614
13616
13655
13677
13

In [23]:
region_id = 5883
region_cells = grid[grid.intersects(regions.loc[region_id].convex_hull)].index

region_cells = region_cells[region_cells.isin(grid.index) & region_cells.isin(grid_assignment.index)]

In [24]:
to_plot = grid.loc[region_cells].to_frame()
to_plot['label'] = grid_assignment.loc[region_cells].values

In [28]:
grid_assignment.to_csv('/data/uscuni-germany/processed_data/cluster_grid_assignment_v2.csv')